In [1]:
import HTMLParser
import re
import itertools
import spacy
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn import feature_extraction
from nltk import Tree

In [2]:
html_parser = HTMLParser.HTMLParser()

In [3]:
%time en_nlp = spacy.load('en')


CPU times: user 7.77 s, sys: 912 ms, total: 8.68 s
Wall time: 17.1 s


In [4]:
def cleaning(original_tweet):
    tweet = html_parser.unescape(original_tweet)
    tweet = original_tweet.decode("utf8").encode('ascii','ignore')
    APPOSTOPHES = {"'s" : " is", "'re" : " are"}
    words = tweet.split()
    reformed = [APPOSTOPHES[word] if word in APPOSTOPHES else word for word in words]
    reformed = " ".join(reformed)
    cleaned = " ".join(re.findall('[A-Z][^A-Z]*', original_tweet))
    tweet = ''.join(''.join(s)[:2] for _, s in itertools.groupby(tweet))
    result = re.sub(r"http\S+", "", tweet)
    s = re.sub("\d+", "", result)
    line = re.sub('[!@#$>&<]', '',s)
    return line
    

In [5]:
class RatedWordgroup:
    def __init__(self,wordgroup,rating):
        self.wordgroup = wordgroup
        self.rating = rating

    def __str__(self):
        result = str(self.wordgroup) +" - " + str(self.rating)
        return result

    def __eq__(self,other):
        return self.wordgroup.eq(other.wordgroup) and self.rating.eq(other.rating)

    def __hash__(self):
        return hash(self.__str__())

In [6]:
class RatedSentence:

    def __init__(self,position,sentence,rating):
        self.position = position
        self.sentence = sentence
        self.rating = rating

    def __str__(self):
        result = "(" + str(self.position) +") (" + str(self.rating) +") " + self.sentence
        return result

    def __eq__(self,other):
        return self.position.eq(other.position) and self.sentence.eq(other.sentence) and self.rating.eq(other.rating)

    def __hash__(self):
        return hash(self.rating)

In [7]:
def get_word_bag(text) :
    text = text.lower()
    text = re.sub("[^a-ž]+", " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub("[ ]+", " ", text)
    wordlist = text.split(" ")
    word_bag = list()
    for word in wordlist :
        word_bag.append(word.strip())
    return word_bag

In [8]:
def get_sentences(text,delimiter='.') :
    text = re.sub("\n", " ", text)
    text = re.sub("[ ]+", " ", text)
    sentences = text.split(delimiter)
    return sentences

In [9]:
def group_words(text) :
    word_bag = get_word_bag(text)

    cleanset = set()
    used_words = set()
    word_groups = list()

    for word in word_bag :
        cleanset.add(word.strip())

    for word in cleanset :
        if word not in used_words :
            wordfamily = list()
            wordfamily.append(word)
            if(len(word)>2) :
                used_words.add(word)
                for other in cleanset :
                    if other not in used_words :
                        same_chars = 0
                        (min_len,max_len) = (len(word),len(other)) if len(word) <= len(other) else (len(other),len(word))
                        for x in range(0,min_len) :
                            if word[x] == other[x] :
                                same_chars += 1
                            else :
                                break
                        if same_chars > 3 and (max_len - same_chars) < 7 :
                            wordfamily.append(other)
                            used_words.add(other)
            word_groups.append(wordfamily)
    return word_groups

In [10]:
def get_wordlist_rate(text) :
    word_groups = group_words(text)
    word_bag = get_word_bag(text)
    rated_word_set = set()
    for group in word_groups :
        occur = 0
        for word in group :
            occur += word_bag.count(word)
        rated_word_set.add(RatedWordgroup(group,occur))
    return rated_word_set

In [11]:
def rate_sentences(text,percentage=10,verbose=True) :
    result = str()
    sentences = get_sentences(text)
    wordlist = get_wordlist_rate(text)
    rated = list()
    topwords = list()
    sort_wg = sorted(wordlist, key = lambda group : group.rating, reverse=True)
    for rwg in sort_wg:
        if rwg.rating > 1 and len(rwg.wordgroup[0]) > 4 :
            #and len(rwg.wordgroup) > 1:
            weight = 1
            weight += rwg.rating/2
            weight += len(rwg.wordgroup[0])/2
            weight += len(rwg.wordgroup)/1
            for word in rwg.wordgroup :
                topwords.append((weight,word))
    position = 0
    for sentence in sentences :
        rating = 0
        position += 1
        bag = get_word_bag(sentence)
        for word in bag :
            for record in topwords :
                 if word.lower()==record[1]:
                    rating += record[0]
        if(rating>0 and len(bag) > 0):
            if((len(bag)/7) !=0):
                rating = rating/((len(bag))/7)
            else:
                rating=0

        rated_sentence = RatedSentence(position, sentence, rating)

        rated.append(rated_sentence)

    sort_sentences = sorted(rated, key = lambda sen : sen.rating, reverse=True)
    num_of_sen = int((len(sentences) / 100.0) * percentage)
    unsorted_result = list()

    counter = 0
    for rs in sort_sentences:
        if(counter>num_of_sen):
            break
        else:
            unsorted_result.append(rs)
            counter += 1

    sort_result = sorted(unsorted_result, key = lambda sen : sen.position, reverse=False)

    for rs in sort_result:
        result += rs.sentence + "."

    return result

In [14]:
synopses_wiki = open('data/synopses_list_wiki.txt').read().split('\n BREAKS HERE')
print("Number of synopsis  ----")
len(synopses_wiki)

Number of synopsis  ----


101

In [15]:
synopses_wiki = synopses_wiki[:100]

In [20]:
suma=[]

for i in synopses_wiki:
    suma.append(cleaning(i))

In [21]:
sumar=[]

for i in suma:
    get_sentences(i)
    get_wordlist_rate(i)
    sumar.append(rate_sentences(i))

" Sollozzo kidnaps Hagen to pressure Sonny to accept his deal. Sollozzo kidnaps Hagen to pressure Sonny to accept his deal. Michael takes refuge in Sicily, and Fredo Corleone is sheltered by associate Moe Greene in Las Vegas. Michael's time abroad has led to marriage to Apollonia Vitelli. Michael's time abroad has led to marriage to Apollonia Vitelli. He denies killing Carlo when questioned by Kay, an answer she accepts. He denies killing Carlo when questioned by Kay, an answer she accepts."

In [22]:
from nltk.corpus import stopwords
def stopword_removal(text1):
    tex=text1.split()
    text = ' '.join([word for word in tex if word not in stopwords.words('english')])
    return text

In [24]:
stopsum=[]
for i in sumar:
    stopsum.append(stopword_removal(i))

In [25]:
stemmer = SnowballStemmer("english")

In [26]:
def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [27]:
ts=[]
tj=[]
for i in stopsum:
    ts.append(tokenize_and_stem(i))
    
for i in ts:
    tj.append(' '.join(i))
    

In [31]:
doc=[]

for i in tj:
    doc.append(en_nlp(i.decode('utf-8')))

In [33]:
class Matgra:
    
    def __init__(self, dc,a):
        self.dG=a
        self.doc=dc
        
    def to_nod(self,node):
        if not self.dG.has_node(node):
            self.dG.add_node(node)
        if node.n_lefts + node.n_rights > 0:
            for child in node.children:
                self.dG.add_edge(node,child)
                self.to_nod(child)
        
    def to_send(self):
        [self.to_nod(sent.root) for sent in self.doc.sents]
        
    def to_res(self):
        self.to_send()
        a=nx.to_numpy_matrix(self.dG)
        return a

In [34]:
import networkx as nx
import matplotlib.pyplot as plt

In [36]:
res=[]

for i in doc:
    dG = nx.DiGraph()
    b=Matgra(i,dG)
    res.append(b.to_res())
    

matrix([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [37]:
import numpy as np

In [38]:
class Graph:
    s=0
    def __init__(self, graph):
        self.graph = np.asarray(graph)
        self.graphSize = graph.shape[0]
        s=graph.shape[0]
        self.nodeList=[]
        self.inDegreeNodeList = [[] for _ in range(s)]
        self.outDegreeNodeList = [[] for _ in range(s)]
     
    def setInDegreeNodeList(self):
        for i in range(0, self.graphSize):
            for j in range(0, self.graphSize):
                if self.graph[i][j] != 0:
                    self.inDegreeNodeList[j].append(i)
    
            
    def setOutDegreeNodeList(self): 
        for i in range(0, self.graphSize):
            for j in range(0, self.graphSize):
                if self.graph[i][j] != 0:
                    self.outDegreeNodeList[i].append(j)
            
    def getGraph(self):
        return self.graph

    def getGraphSize(self):
        return self.graphSize

    def getInDegreeNodeList(self):
        self.setInDegreeNodeList()
        return self.inDegreeNodeList

    def getOutDegreeNodeList(self):
        self.setOutDegreeNodeList()
        return self.outDegreeNodeList
    
    def setNodeList(self):
        i = 0
        while i < self.graphSize: 
            self.nodeList.append(i)
            i += 1
            
    def getNodeList(self):
        self.setNodeList()
        return self.nodeList

In [39]:
gr=[]

for i in res:
    gr.append(Graph(i))


In [42]:
class NMSimilarity():
    ga=0
    gb=0
    def __init__(self, graphA, graphB, epsilon):
        self.graphA = graphA
        self.graphB = graphB
        self.epsilon = epsilon
        self.inNodeListA = graphA.getInDegreeNodeList()
        self.outNodeListA = graphA.getOutDegreeNodeList()
        self.inNodeListB = graphB.getInDegreeNodeList()
        self.outNodeListB = graphB.getOutDegreeNodeList()
        self.graphSizeA = graphA.getGraphSize()
        ga=graphA.getGraphSize()
        self.graphSizeB = graphB.getGraphSize()
        gb=graphB.getGraphSize()
        self.nodeSimilarity = [[0 for x in range(ga)] for y in range(gb)]
        self.inNodeSimilarity = [[0 for x in range(ga)] for y in range(gb)]
        self.outNodeSimilarity = [[0 for x in range(ga)] for y in range(gb)]
        
        
    def initializeSimilarityMatrices(self):
        for i in range(0,self.graphSizeA):
            for j in range(0,self.graphSizeB):
                maxDegree = float(max(len(self.inNodeListA[i]),len(self.inNodeListB[j])))
                if maxDegree != 0:
                    self.inNodeSimilarity[i][j] = ((min(len(self.inNodeListA[i]), len(self.inNodeListB[j]))) / (maxDegree))
                else:
                    self.inNodeSimilarity[i][j] = float(0.0)


                maxDegree = float(max(len(self.outNodeListA[i]), len(self.outNodeListB[j])));
                if maxDegree != 0:
                    self.outNodeSimilarity[i][j] = ((min(len(self.outNodeListA[i]), len(self.outNodeListB[j]))) / (maxDegree))
                else:
                    self.outNodeSimilarity[i][j] = float(0.0)

        for i in range(0,self.graphSizeA):
            for j in range(0,self.graphSizeB) :
                self.nodeSimilarity[i][j] = (self.inNodeSimilarity[i][j] + self.outNodeSimilarity[i][j])
        
    def enumerationFunction(self, neighborListMin, neighborListMax, graph):
        similaritySum = 0.0
        valueMap = {}
        if graph == 0:
            l1=len(neighborListMin)
            for i in range(0,l1):
                node=neighborListMin[i]
                maxi=0.0
                maxIndex=-1
                l2=len(neighborListMax)
                for j in range(0,l2):
                    key=neighborListMax[j]
                    if key not in valueMap:
                        try:
                            if maxi < self.nodeSimilarity[node][key]:
                                maxi = self.nodeSimilarity[node][key]
                                maxIndex = key
                        except IndexError:
                            pass
                        continue
                valueMap[maxIndex]=maxi
        else:
            for i in range(len(neighborListMin)):
                node=neighborListMin[i]
                maxi=0.0
                maxIndex=-1
                for j in range(len(neighborListMax)):
                    key=neighborListMax[j]
                    if key not in valueMap:
                        try:
                            if maxi < self.nodeSimilarity[key][node]:
                                maxi = self.nodeSimilarity[key][node]
                                maxIndex = key
                        except IndexError:
                            pass
                        continue
                valueMap[maxIndex]=maxi
        valu=0.0       
        for val in valueMap.itervalues():
            valu=valu+float(val)
        similaritySum=valu
        return similaritySum
        
        
        
    def measureSimilarity(self):
        maxDifference = 0.0
        terminate = False
        while not terminate:
            maxDifference = 0.0
            for i in range(0, self.graphSizeA):
                for j in range(0, self.graphSizeB):
                    similaritySum = 0.0
                    try:
                        maxDegree = float(max(len(self.inNodeListA[i]), len(self.inNodeListB[j])))
                        minDegree = int(min(len(self.inNodeListA[i]), len(self.inNodeListB[j])))
                        # calculate in-degree similarities
                        if minDegree == len(self.inNodeListA[i]):
                            similaritySum = self.enumerationFunction(self.inNodeListA[i], self.inNodeListB[j], 0)
                        else:
                            similaritySum = self.enumerationFunction(self.inNodeListB[j], self.inNodeListA[i], 1)
                        if maxDegree == 0.0 and similaritySum == 0.0:
                            self.inNodeSimilarity[i][j] = 1.0
                        elif maxDegree == 0.0:
                            self.inNodeSimilarity[i][j] = 0.0
                        else:

                            self.inNodeSimilarity[i][j]=float(similaritySum / maxDegree)
                    except IndexError:
                        pass
                    continue
                    similaritySum = 0.0
                    try:
                        maxDegree = float(max(len(self.outNodeListA[i]), len(self.outNodeListB[j])));
                        minDegree = int(min(len(self.outNodeListA[i]), len(self.outNodeListB[j])));
                        if minDegree == len(self.outNodeListA[i]):
                            similaritySum = self.enumerationFunction(self.outNodeListA[i], self.outNodeListB[j], 0)
                        else:
                            similaritySum = self.enumerationFunction(self.outNodeListB[j], self.outNodeListA[i], 1)
                        if maxDegree == 0.0 and similaritySum == 0.0:
                            self.outNodeSimilarity[i][j] = 1.0
                        elif maxDegree == 0.0:
                            self.outNodeSimilarity[i][j] = 0.0
                        else:
                            self.outNodeSimilarity[i][j] = similaritySum / maxDegree
                    except IndexError:
                        pass
                    continue


            for i in range(0,self.graphSizeA):
                for j in range(0,self.graphSizeB):
                    try:
                        temp = (self.inNodeSimilarity[i][j] +self.outNodeSimilarity[i][j]);
                        if abs(self.nodeSimilarity[i][j] - temp) > maxDifference:
                            maxDifference = abs(self.nodeSimilarity[i][j] - temp)
                        self.nodeSimilarity[i][j] = temp
                    except IndexError:
                        pass
                    continue


            if maxDifference < self.epsilon:
                terminate = True
 
    def getGraphSimilarity(self):
        finalGraphSimilarity = 0.0
        self.measureSimilarity()
        
        if self.graphA.getGraphSize() < self.graphB.getGraphSize():
            finalGraphSimilarity = self.enumerationFunction(self.graphA.getNodeList(), self.graphB.getNodeList(), 0) / self.graphA.getGraphSize()
        else:
            finalGraphSimilarity = self.enumerationFunction(self.graphB.getNodeList(), self.graphA.getNodeList(), 1) / self.graphB.getGraphSize()
        finalGraphSimilarity = float('{:f}'.format(finalGraphSimilarity * 100))
        return finalGraphSimilarity

In [43]:
nm=NMSimilarity(gr[11],gr[95],0.0001)
nm.getGraphSimilarity()

53.846154

In [44]:
int(round(nm.getGraphSimilarity()))

54

In [ ]:
aten=np.zeros((11,11))

In [ ]:
for i in range(0,11):
    for j in range(0,11):
        if i!=j:
            nm=NMSimilarity(gr[i],gr[j],0.0001)
            aten[i][j]=int(nm.getGraphSimilarity())

In [73]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(aten)
kmeans.labels_

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)